In [ ]:
!git clone https://github.com/youssefboutaleb/colorizeblack-whitephotos
import os
os.listdir("/content/colorizeblack-whitephotos")


In [ ]:
#import libs
from tensorflow.keras.utils import img_to_array,array_to_img,load_img
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator 
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf
import matplotlib.pyplot as plt



In [ ]:
# Get images

import random
X = []
#choose a random folder 
data="Train"+str(random.randint(0, 7))
print(data)
for filename in os.listdir('/content/colorizeblack-whitephotos/Beta-version/Train/'+data):
    X.append(img_to_array(load_img('/content/colorizeblack-whitephotos/Beta-version/Train/'+data+"/"+filename)))
X = np.array(X, dtype=float)
# Set up training and test data
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain

Train6


In [ ]:

#Design the neural network
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))

In [ ]:
# Finish model
model.compile(optimizer='rmsprop', loss='mse')


In [ ]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)
# Generate training data
batch_size = 50
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

In [ ]:
best_loss = float('inf') 
best_params = None
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128
#find Best parameters
for batch_size in [32, 64, 128]:
    for epochs in [10, 15, 20]:
        for steps_per_epoch in [500, 700, 900]:
            print(f'Trying batch_size={batch_size}, epochs={epochs}, steps_per_epoch={steps_per_epoch}')
            
            
            model.fit(image_a_b_gen(batch_size), 
                                steps_per_epoch=steps_per_epoch, epochs=epochs)
                                
            val_loss = model.evaluate(Xtest, Ytest, batch_size=batch_size)
            if val_loss < best_loss:
                best_loss = val_loss 
                best_params = {'batch_size': batch_size, 
                              'epochs': epochs,
                              'steps_per_epoch': steps_per_epoch}

print(f'Best parameters: {best_params} with validation loss={best_loss}')